In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Đọc danh sách URL từ tệp
url_list = []
with open('http.csv', 'r') as file:
    url_list = [line.strip() for line in file]

# Tiêu đề User-Agent giả lập một trình duyệt
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Khởi tạo danh sách để lưu thông tin sản phẩm
all_product_list = []

# Định nghĩa hàm để lấy danh mục từ URL
def get_category_from_url(url):
    if 'dtdd' in url:
        return 'Điện thoại'
    elif 'may-tinh-bang' in url:
        return 'Máy tính bảng'
    elif 'laptop' in url:
        return 'Máy tính xách tay'
    elif 'dong-ho-thong-minh' in url:
        return 'Đồng hồ thông minh'
    return 'Khác'

# Duyệt qua từng URL và lấy thông tin
for url in url_list:
    # Lấy tên hãng từ URL
    sub_category = url.split('-')[-1].capitalize()
    category = get_category_from_url(url)
    
    # Gửi yêu cầu đến trang web với tiêu đề User-Agent và thời gian chờ (timeout)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # Kiểm tra lỗi

    # Phân tích HTML của trang web
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm các thẻ chứa thông tin sản phẩm
    products = soup.find_all('li', class_='item')

    # Duyệt qua từng sản phẩm và lấy thông tin
    for product in products:
        try:
            # Kiểm tra và tìm thẻ chứa tên sản phẩm
            title_tag = product.find('h3')
            title = title_tag.text.strip() if title_tag else 'No title available'

            # Kiểm tra và tìm thẻ chứa giá sản phẩm
            price_tag = product.find('strong')
            price = price_tag.text.strip().replace('₫', '').strip() if price_tag else 'No price available'

            # Kiểm tra và tìm thẻ chứa URL ảnh
            image_url = 'No image URL'
            img_div = product.find('div', class_='item-img')
            if img_div:
                img_tag = img_div.find('img')
                if img_tag and 'data-src' in img_tag.attrs:
                    image_url = img_tag['data-src']
                elif img_tag and 'src' in img_tag.attrs:
                    image_url = img_tag['src']

            # Lấy thông tin chi tiết sản phẩm từ phần mô tả ngắn
            description_tag = product.find('div', class_='utility')
            if description_tag:
                description_items = description_tag.find_all('p')
                description = ' | '.join(item.text.strip() for item in description_items) if description_items else 'No description available'
            else:
                description = 'No description available'

            # Lưu thông tin sản phẩm vào danh sách
            all_product_list.append({
                'category': category,
                'sub_category': sub_category,
                'title': title,
                'description': description,
                'price': price,
                'image_url': image_url
            })
        except Exception as e:
            print(f"Error processing product: {e}")
            continue

# Tạo DataFrame từ danh sách sản phẩm
df = pd.DataFrame(all_product_list)

# Lọc bỏ các sản phẩm không hợp lệ
df = df[df['title'] != 'No title available']
df = df[df['price'] != 'No price available']
df = df[df['image_url'] != 'No image URL']
df = df[df['description'] != 'No description available']

# Loại bỏ các sản phẩm trùng lặp dựa trên cột 'title'
df = df.drop_duplicates(subset=['title'])

# Lưu DataFrame vào file CSV
output_path = 'products.csv'
df.to_csv(output_path, index=False)

print(df.head(20))  # In 20 sản phẩm đầu tiên để kiểm tra


      category sub_category                        title  \
0   Điện thoại       Iphone            iPhone 15 Pro Max   
4   Điện thoại       Iphone                    iPhone 13   
7   Điện thoại       Iphone                iPhone 15 Pro   
12  Điện thoại       Iphone               iPhone 15 Plus   
16  Điện thoại       Iphone                    iPhone 15   
19  Điện thoại       Iphone                    iPhone 11   
22  Điện thoại       Iphone                    iPhone 12   
25  Điện thoại       Iphone            iPhone 14 Pro Max   
30  Điện thoại       Iphone               iPhone 14 Plus   
33  Điện thoại       Iphone                    iPhone 14   
36  Điện thoại      Samsung  Samsung Galaxy S23 Ultra 5G   
39  Điện thoại      Samsung        Samsung Galaxy A55 5G   
43  Điện thoại      Samsung        Samsung Galaxy M54 5G   
44  Điện thoại      Samsung     Samsung Galaxy S23 FE 5G   
47  Điện thoại      Samsung        Samsung Galaxy A35 5G   
50  Điện thoại      Samsung  Samsung Gal